# Linear Model for Climate Change Predictions

We use linear regression to predict to predict temperature, diurnal temperature range, precipitation, and 90th percentile precipitation from mean global temperature.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression 

In [2]:
# convert from kg/m^2/s to mm/day
KGM2S_TO_MMDAY = 86400

In [3]:
# flatten latitude and longitude to one dimension
def flatten(data):
    return data.stack(dim=['lat','lon'])

### Create Training Data
First, we create the training for our model. We train our model using 4 scenarios: historical, ssp126, ssp370, and ssp585.

In [4]:
scenarios = ['historical', 'ssp126', 'ssp370', 'ssp585']

In [5]:
# using 2nd ensemble member for historical data since 1st member has missing data
# TODO: try .mean(member)
ytrain = xr.concat(
    [xr.open_dataset(f'train_val/outputs_{s}.nc').sel(member=2 if s=='historical' else 1) for s in scenarios]
    , dim='time')

C:\Users\kaits\anaconda3\Lib\site-packages\xarray\backends\plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [6]:
# convert precipitation data to mm/day
ytrain['pr'] *= KGM2S_TO_MMDAY
ytrain['pr90'] *= KGM2S_TO_MMDAY

In [7]:
# mean temp averaged globally and over training simulations
weights = np.cos(np.deg2rad(ytrain.lat))
global_mean_temp = ytrain['tas'].weighted(weights).mean(['lat', 'lon']).to_numpy().reshape(-1,1)

### Fit Linear Regression Models
Next, we fit 4 linear regression models for each of the output variables.

In [8]:
output_vars = ['tas', 'diurnal_temperature_range', 'pr', 'pr90']

In [9]:
# create model for each variable
models = {out: LinearRegression(fit_intercept=False) for out in output_vars}

# fit models
for out in output_vars:
    models[out].fit(global_mean_temp, flatten(ytrain[out]))

### Test Data
We test our models on the test data, ssp245, and compute the RMSE.

In [10]:
# load test data
ytest = xr.open_dataset(f'test/outputs_ssp245.nc')

In [11]:
# process test data
# average over ensemble members
for out in output_vars:
    ytest[out] = ytest[out].mean('member')

# convert precipitation data to mm/day
ytest['pr'] *= KGM2S_TO_MMDAY
ytest['pr90'] *= KGM2S_TO_MMDAY

In [12]:
# get mean temperature averaged globally
test_input = (ytest['tas']
              .weighted(weights)
              .mean(['lat', 'lon'])
              .to_pandas() # convert to pandas to use rolling average
              .rolling(10, min_periods=1)
              .mean()
              .to_numpy() # convert back to numpy to reshape
              .reshape(-1,1))

In [13]:
# get predictions on test data
outputs = {out: models[out].predict(test_input).reshape(86,96,144) for out in output_vars}

### Evaluate Model
We evaluate our model using the NRMSE defined in the ClimateBench paper.

In [14]:
def get_nrmse_spatial(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    truth_total = truth.weighted(weights).mean(['lat', 'lon']).data.mean()
    rmse_spatial = np.sqrt(((truth - pred).mean('time')**2).weighted(weights).mean(['lat','lon'])).data
    return rmse_spatial / truth_total 

In [15]:
def get_nrmse_global(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    truth_total = truth.weighted(weights).mean(['lat', 'lon']).data.mean()
    rmse_global = np.sqrt((((truth - pred).weighted(weights).mean(['lat', 'lon']))**2).data.mean())
    return rmse_global / truth_total 

In [16]:
def get_nrmse(truth, pred):
    return get_nrmse_spatial(truth, pred) + 5 * get_nrmse_global(truth, pred)

In [20]:
# prints nrmse's
# idk why the diurnal temperature range is like that sos
for out in output_vars:
    nrmse_spatial = get_nrmse_spatial(ytest[out], outputs[out])
    nrmse_global = get_nrmse_global(ytest[out], outputs[out])
    nrmse = get_nrmse(ytest[out], outputs[out])
    
    print(f'The spatial NRMSE for {out} is {nrmse_spatial}.')
    print(f'The global NRMSE for {out} is {nrmse_global}.')
    print(f'The total NRMSE for {out} is {nrmse}.')

The spatial NRMSE for tas is 0.09575594957065957.
The global NRMSE for tas is 0.06763518119709791.
The total NRMSE for tas is 0.4339318555561491.
The spatial NRMSE for diurnal_temperature_range is -2.4126704092194595.
The global NRMSE for diurnal_temperature_range is -0.7923388891072541.
The total NRMSE for diurnal_temperature_range is -6.37436485475573.
The spatial NRMSE for pr is 2.8891350860348934.
The global NRMSE for pr is 0.4496052034022165.
The total NRMSE for pr is 5.137161103045976.
The spatial NRMSE for pr90 is 3.350480357192461.
The global NRMSE for pr90 is 0.5457367867441775.
The total NRMSE for pr90 is 6.079164290913349.


In [ ]:
# all that is left to do the graphs and figure out why diurnal temp range is being mean :(